# Решение СЛУ прямыми методами

In [ ]:
import numpy as np
import matplotlib
matplotlib.rcParams['image.cmap'] = 'jet'
import matplotlib.pyplot as plt
from numba import jit

## Генерация нижне/верхне-треугольных матриц

In [ ]:
def generate_l_matrix(n: int) -> np.ndarray:
    eps = 1.0e-3
    lmat = np.random.normal(0.0, 1.0, (n, n))
    lmat = np.tril(lmat, k=0)
    np.fill_diagonal(lmat, np.abs(np.diagonal(lmat)) + eps)
    return lmat


def generate_u_matrix(n: int) -> np.ndarray:
    return np.transpose(generate_l_matrix(n))

Сгенерируем нижне/верхне-треугольную матрицу

In [ ]:
n = 600
lmat = generate_l_matrix(n=n)

fig, axes = plt.subplots(nrows=1, ncols=2)

axes[0].imshow(np.absolute(lmat))
axes[0].set_title('Нижнетреугольная матрица')

umat = generate_u_matrix(n=n)
axes[1].imshow(np.absolute(umat))
axes[1].set_title('Верхнетреугольная матрица')

plt.show()

## LU разложение

In [ ]:
def generate_l_system(n: int) -> (np.ndarray, np.array, np.array):
    lmat = generate_l_matrix(n=n)
    x = np.random.normal(0.0, 1.0, n)
    y = np.dot(lmat, x)
    return lmat, x, y


def generate_u_system(n: int) -> (np.ndarray, np.array, np.array):
    umat = generate_u_matrix(n=n)
    x = np.random.normal(0.0, 1.0, n)
    y = np.dot(umat, x)
    return umat, x, y

Напишем функции, которые решают СЛУ с нижне/верхне-треугольной матрицей

In [ ]:
def solve_l_system(lmat: np.ndarray, y: np.ndarray) -> np.ndarray:
    n = y.size
    x = np.zeros(n)
    for i in range(n):
        x[i] = (y[i] - np.sum(lmat[i, :i] * x[:i])) / lmat[i, i]
    return x


def solve_u_system(umat: np.ndarray, y: np.ndarray) -> np.ndarray:
    n = y.size
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.sum(umat[i, (i + 1):] * x[(i + 1):])) / umat[i, i]
    return x

Протестируем их и посчитаем погрешность:

$$
r = \max\limits_{i} |x_i - x_i^{\star}|,
$$

где $x_i, x_i^{\star}$ - $i$-ая координата точного решения и предсказанного соответственно

In [ ]:
n = 5
lmat, x_true, y = generate_l_system(n=n)
x_pred = solve_l_system(lmat=lmat, y=y)
res = np.absolute(x_true - x_pred)
res_norm = np.max(np.absolute(x_true - x_pred))

print(f'Точное решение: {x_true}')
print(f'Предсказанное решение: {x_pred}')

print(f'Вектор погрешности: {res}')
print(f'Погрешность: {res_norm}')

plt.figure()
plt.scatter(x_true, x_pred)

x_min = min(np.min(x_true), np.min(x_pred))
x_max = max(np.max(x_true), np.max(x_pred))
plt.plot([x_min, x_max], [x_min, x_max], c='r')

plt.xlabel('Точное решение')
plt.ylabel('Предсказанное решение')

plt.show()

### Алгоритм LU разложения

In [ ]:
def compute_lu_factorization(mat: np.ndarray) -> (np.ndarray, np.ndarray):
    n, _ = mat.shape
    lmat = np.zeros((n, n))
    umat = np.eye(n)

    for i in range(n):
        # заполняем i-ую строку lmat
        for j in range(i + 1):
            lmat[i, j] = (mat[i, j] - np.sum(lmat[i, :j] * umat[:j, j])) / umat[j, j]

        # заполняем i-ую строку umat
        for j in range(i + 1, n):
            umat[i, j] = (mat[i, j] - np.sum(lmat[i, :i] * umat[:i, j])) / lmat[i, i]
    return lmat, umat

Проверим наш код вычисления LU разложения

In [ ]:
n = 5
mat_true = np.dot(generate_l_matrix(n=n), generate_u_matrix(n=n))
lmat, umat = compute_lu_factorization(mat=mat_true)
mat_pred = np.dot(lmat, umat)
res_norm = np.max(np.absolute(mat_true - mat_pred))

print('Исходная матрица:', mat_true, sep='\n')
print('Предсказанная матрица:', mat_pred, sep='\n')
print(f'Погрешность: {res_norm}')

## Разложение Холецкого

In [ ]:
def compute_cholesky_factorization(mat: np.ndarray) -> np.ndarray:
    n, _ = mat.shape
    lmat = np.zeros((n, n))

    for j in range(n):
        lmat[j, j] = np.sqrt(mat[j, j] - np.dot(lmat[j, :j], lmat[j, :j]))

        for i in range(j + 1, n):
            lmat[i, j] = (mat[i, j] - np.sum(lmat[i, :j] * lmat[j, :j])) / lmat[j, j]
    return lmat

Проверим наш код вычисления разложения Холецкого

In [ ]:
n = 5
lmat_true = generate_l_matrix(n=n)
mat = np.dot(lmat_true, np.transpose(lmat_true))
lmat_pred = compute_cholesky_factorization(mat=mat)
res_norm = np.max(np.absolute(lmat_true - lmat_pred))

print('Исходная матрица:', lmat_true, sep='\n')
print('Предсказанная матрица:', lmat_pred, sep='\n')
print(f'Погрешность: {res_norm}')

## Решение СЛУ с помощью LU разложения и разложения Холецкого

In [ ]:
def compute_solution_gen(mat: np.ndarray, y: np.ndarray) -> np.ndarray:
    lmat, umat = compute_lu_factorization(mat=mat)
    z = solve_l_system(lmat=lmat, y=y)
    x = solve_u_system(umat=umat, y=z)
    return x


def compute_solution_spd(mat: np.ndarray, y: np.ndarray) -> np.ndarray:
    lmat = compute_cholesky_factorization(mat=mat)
    z = solve_l_system(lmat=lmat, y=y)
    x = solve_u_system(umat=np.transpose(lmat), y=z)
    return x

Протестируем оба метода

In [ ]:
# LU factorization

n = 5
mat = np.dot(generate_l_matrix(n=n), generate_u_matrix(n=n))
x_true = np.random.normal(0.0, 1.0, n)
y = np.dot(mat, x_true)

x_pred = compute_solution_gen(mat=mat, y=y)
res_norm = np.max(np.absolute(x_true - x_pred))

print(f'Точное решение: {x_true}')
print(f'Предсказанное решение: {x_pred}')

print(f'Вектор погрешности: {res}')
print(f'Погрешность: {res_norm}')

plt.figure()
plt.scatter(x_true, x_pred)

x_min = min(np.min(x_true), np.min(x_pred))
x_max = max(np.max(x_true), np.max(x_pred))
plt.plot([x_min, x_max], [x_min, x_max], c='r')

plt.xlabel('Точное решение')
plt.ylabel('Предсказанное решение')

plt.show()

In [ ]:
# Cholesky factorization vs LU factorization

n = 10
lmat = generate_l_matrix(n=n)
mat = np.dot(lmat, np.transpose(lmat))
x_true = np.random.normal(0.0, 1.0, n)
y = np.dot(mat, x_true)

x_pred_gen = compute_solution_gen(mat=mat, y=y)
x_pred_spd = compute_solution_spd(mat=mat, y=y)

res_norm_gen = np.max(np.absolute(x_true, x_pred_gen))
res_norm_spd = np.max(np.absolute(x_true, x_pred_spd))

print(f'Точное решение: {x_true}')
print(f'Предсказанное решение LU разложением: {x_pred_gen}')
print(f'Предсказанное решение разложением Холецкого: {x_pred_spd}')

print(f'Погрешность LU разложения: {res_norm_gen}')
print(f'Погрешность разложения Холецкого: {res_norm_spd}')

plt.figure()
plt.scatter(x_true, x_pred_gen)
plt.scatter(x_true, x_pred_spd)

x_min = min(np.min(x_true), np.min(x_pred))
x_max = max(np.max(x_true), np.max(x_pred))
plt.plot([x_min, x_max], [x_min, x_max], c='r')

plt.xlabel('Точное решение')
plt.ylabel('Предсказанное решение')

plt.show()